    !pip install stopit

# Benchmark the reusability of literate coding documents.

In [25]:
    from collections import defaultdict
    from importnb import Notebook
    from pathlib import Path 
    from stopit import ThreadingTimeout
    import os, sys
    import pandas as pd, importlib
    from IPython import get_ipython
    from IPython.utils.capture import capture_output
    from tqdm import tqdm_notebook as progress
    from ricker import TqdmAudioTicker as progress
    import pandas
    from toolz.curried import *; from toolz.curried.operator import *; dunder="__%s__"
    from hypothesis import given, strategies

In [2]:
    timeout = .1
    projects = 'PythonDataScienceHandbook/notebooks/', 'ThinkDSP/code/'

In [26]:
    def load_notebooks_from_a(project):
        global timeout
        current = os.getcwd()
        name = Path(project).parent.name
        !git clone --depth 1 $name 
        try:
            notebooks = list(Path(project).glob('*.ipynb'))
            bar = progress(notebooks)
            for notebook in bar:
                bar.set_description(notebook.name)
                with ThreadingTimeout(timeout), capture_output() as captured:
                    try: module = Notebook.load(notebook); module.__output__ = captured
                    except BaseException as e: 
                        module = importlib.util.module_from_spec(importlib.machinery.ModuleSpec(Path(notebook).stem, str(notebook), origin=str(notebook)))
                        with importlib._bootstrap._installed_safely(module): 
                            module.__file__ = str(notebook)
                            module.__exception__ = e
                os.chdir(current)
        except KeyboardInterrupt:...
        finally: os.chdir(current)

    _load_notebooks_from_a = given(project=strategies.one_of(*map(strategies.just, projects)))(load_notebooks_from_a)

In [18]:
    __name__ == '__main__' and _load_notebooks_from_a()

fatal: destination path 'PythonDataScienceHandbook' already exists and is not an empty directory.


fatal: destination path 'ThinkDSP' already exists and is not an empty directory.


Turn the contents of existing projects into dataframes based on a project directories.

In [27]:
    def project_frames(*projects):
        s = pd.Series(sys.modules)
        all_the_files = sum((list(Path(object).rglob('*.ipynb')) for object in projects), [])
        return s[s.apply(
            lambda x: Path(getattr(x, '__file__', '') or '') in all_the_files
        ).dropna()].apply(compose(pd.Series, vars))

In [28]:
    def _extract_dataframes_from_sys_modules():
        assert len(project_frames(*projects))